# GDMA - Assignment 1

Autor: Julian Schelb (1069967)

## Imports

In [1]:
from neo4j import GraphDatabase
import pandas as pd
import numpy as np

***

## Database Connection

In [2]:
class Neo4jConnection:
    
    # Create connection driver object
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
    
    # Close Connection
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
    
    # Query database
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [3]:
    conn = Neo4jConnection(uri="bolt://localhost:7687", user="pythonuser", pwd="")

***

## Task 1

For each supplier in the UK, print the names of all condiments in a list,
their average, their minimum, and their maximum price.

In [5]:
query_string = "MATCH (n1:Supplier)-[r:SUPPLIES]->(n2:Product) \
WHERE n2.categoryID = '2' \
RETURN n1.companyName as company, \
collect(n2.productName) as products, \
AVG(n2.unitPrice) as avg_price, \
MIN(n2.unitPrice) as min_price, \
max(n2.unitPrice) as max_price" 

In [6]:
dtf_data = pd.DataFrame([dict(_) for _ in conn.query(query_string, db="northwind-graph")])
dtf_data

,company,products,avg_price,min_price,max_price
0,Exotic Liquids,[Aniseed Syrup],10.00,10.00,10.00
1,New Orleans Cajun Delights,"[Chef Anton's Cajun Seasoning, Chef Anton's Gu...",20.35,17.00,22.00
2,Grandma Kelly's Homestead,"[Grandma's Boysenberry Spread, Northwoods Cran...",32.50,25.00,40.00
3,Mayumi's,[Genen Shouyu],15.50,15.50,15.50
4,Leka Trading,[Gula Malacca],19.45,19.45,19.45
5,Forêts d'érables,[Sirop d'érable],28.50,28.50,28.50
6,Pavlova,[Vegie-spread],43.90,43.90,43.90
7,Plutzer Lebensmittelgroßmärkte AG,[Original Frankfurter grüne Soße],13.00,13.00,13.00


***

## Task 2

Compute the cost of the order that consists of the entire stock of products
supplied by the company ’Exotic Liquids’.

In [7]:
query_string = "MATCH (n1:Supplier)-[r:SUPPLIES]->(n2:Product) \
WHERE n1.companyName = 'Exotic Liquids' \
RETURN  n1.companyName as company, SUM(n2.unitPrice * n2.unitsInStock) as total_order_price"

In [8]:
dtf_data = pd.DataFrame([dict(_) for _ in conn.query(query_string, db="northwind-graph")])
dtf_data

,company,total_order_price
0,Exotic Liquids,1155.0


***

## Task 3

Print the names and the full addresses of all suppliers. A full address
should contain the address, the postal code and the country and must be
returned as a single string (you can concatenate strings with ’+’, don’t
worry if some values are null or don’t make sense). For each supplier also
print the number of orders they have received from users.

In [9]:
query_string = "MATCH (n1:Order)-[r]->(n2:Product), \
(n3:Supplier)-[r2]->(n2:Product) \
RETURN n3.companyName as company,  \
n3.address + ' ' + n3.postalCode + ' ' + n3.country as company_address, \
count(n1) as count_orders"

In [10]:
dtf_data = pd.DataFrame([dict(_) for _ in conn.query(query_string, db="northwind-graph")])
dtf_data

,company,company_address,count_orders
0,Exotic Liquids,49 Gilbert St. EC1 4SD UK,94
1,New Orleans Cajun Delights,P.O. Box 78934 70117 USA,70
2,Grandma Kelly's Homestead,707 Oxford Rd. 48104 USA,54
3,Tokyo Traders,9-8 Sekimai Musashino-shi 100 Japan,51
4,Cooperativa de Quesos 'Las Cabras',Calle del Rosal 4 33007 Spain,52
5,Mayumi's,92 Setsuko Chuo-ku 545 Japan,68
6,Pavlova,Marketing Manager Victoria 3058,163
7,Specialty Biscuits,Sales Representative NULL M14 GSD,126
8,PB Knäckebröd AB,Kaloadagatan 13 S-345 67 Sweden,34
9,Refrescos Americanas LTDA,Av. das Americanas 12.890 5442 Brazil,51


***

## Task 4

Find all pairs of suppliers that sell products of exactly the same categories
(use the categoryID for comparisons) and print their names.


In [11]:
query_string = "MATCH (s1:Supplier)-[r1:SUPPLIES]->(p1:Product), \
(s2:Supplier)-[r2:SUPPLIES]->(p2:Product) \
WITH s1, s2, p1, p2 \
ORDER BY p1.categoryID, p2.categoryID \
WHERE s1 <> s2 \
WITH s1.companyName as s1_name, s2.companyName as s2_name, \
collect(DISTINCT p1.categoryID) as p1_ids, \
collect(DISTINCT p2.categoryID) as p2_ids \
WHERE p1_ids = p2_ids \
RETURN \
DISTINCT CASE WHEN s1_name < s2_name THEN s1_name + ' & ' + s2_name ELSE s2_name + ' & ' + s1_name END as pair"

In [12]:
dtf_data = pd.DataFrame([dict(_) for _ in conn.query(query_string, db="northwind-graph")])
dtf_data

,pair
0,Bigfoot Breweries & Refrescos Americanas LTDA
1,Aux joyeux ecclésiastiques & Bigfoot Breweries
2,Aux joyeux ecclésiastiques & Refrescos America...
3,Heli Süßwaren GmbH & Co. KG & Specialty Biscuits
4,Specialty Biscuits & Zaanse Snoepfabriek
5,Heli Süßwaren GmbH & Co. KG & Zaanse Snoepfabriek
6,Cooperativa de Quesos 'Las Cabras' & Formaggi ...
7,Cooperativa de Quesos 'Las Cabras' & Norske Me...
8,Cooperativa de Quesos 'Las Cabras' & Gai pâturage
9,Formaggi Fortini s.r.l. & Gai pâturage


***

## Task 5

For a given productID, find how many times the product has been sold
per year for the years 1996–1998.

In [13]:
query_string = "MATCH (n1:Product)<-[r]-(n2:Order) \
WHERE n1.productID = '10' \
    AND n2.orderDate >= '1996-01-01 00:00:00.000' \
    AND n2.orderDate <  '1999-01-01 00:00:00.000' \
RETURN  DISTINCT left(n2.orderDate,4) as year, count(n2.orderID) as orders"

In [14]:
dtf_data = pd.DataFrame([dict(_) for _ in conn.query(query_string, db="northwind-graph")])
dtf_data

,year,orders
0,1996,4
1,1997,19
2,1998,10


***

## Task 6

Find the customer that has spent the highest amount of money on orders
in October 1997.

In [15]:
query_string = "MATCH (n2:Customer)-[r]->(n1:Order), \
(n3:Product)<-[r2]-(n1:Order) \
WHERE left(n1.orderDate, 7) = '1997-10' \
RETURN n2.companyName as company, \
sum((toFloat(r2.unitPrice) * r2.quantity * ( 1 - toFloat(r2.discount))) + toFloat(n1.freight)) as total_discounted_and_freight, \
sum(toFloat(r2.unitPrice) * r2.quantity * ( 1 - toFloat(r2.discount))) as total_discounted, \
sum(toFloat(r2.unitPrice) * r2.quantity) as total \
ORDER BY total_discounted DESC \
LIMIT 1"

In [16]:
dtf_data = pd.DataFrame([dict(_) for _ in conn.query(query_string, db="northwind-graph")])
dtf_data

,company,total_discounted_and_freight,total_discounted,total
0,QUICK-Stop,21207.925,15913.675,15962.3


*** 

## Task 7 

 For every country, print all suppliers, along with the total value of the
products they have in stock. Make sure that you print the name of the
country only if its the name of a real country (if you check the names of
the countries you will understand). Otherwise, instead of the country’s
name, print null.

In [17]:
query_string = "MATCH (n1:Supplier) -[r:SUPPLIES]->(n2:Product) \
RETURN CASE WHEN toFloat(n1.country) is not null THEN 'null' ELSE n1.country END as country, SUM(n2.unitsInStock * n2.unitPrice) as in_stock_value \
ORDER BY in_stock_value"

In [18]:
dtf_data = pd.DataFrame([dict(_) for _ in conn.query(query_string, db="northwind-graph")])
dtf_data

,country,in_stock_value
0,Brazil,90.00
1,Netherlands,533.25
2,UK,1155.00
3,Denmark,1187.50
4,Norway,1775.00
5,Finland,2282.25
6,M14 GSD,3575.00
7,Spain,3730.00
8,Japan,5376.25
9,Sweden,6662.00


***

## Task 8

Write a query that adds a relationship of type :REGULAR between a customer and a supplier, if the customer has ordered more than 5 times at
least one product from the supplier. The query must also return the customer name and the the company name of the supplier.

In [19]:
#MATCH (n1:Supplier)-[r]->(n2:Product)<-[r2]-(n3:Order)<-[r3]-(n4:Customer)
#WITH n1.companyName as company,  n4.contactName as customer, count(n4) as order_count
#WHERE order_count > 4
#RETURN company, customer, order_count
#LIMIT 100

In [20]:
query_string = "MATCH (n1:Supplier)-[r]->(n2:Product)<-[r2]-(n3:Order)<-[r3]-(n4:Customer) \
WITH n1.companyName as company,  n4.contactName as customer, count(n4) as order_count \
WHERE order_count > 4 \
CREATE (n1)<-[:REGULAR]-(n2) \
RETURN company, customer, order_count"

In [21]:
dtf_data = pd.DataFrame([dict(_) for _ in conn.query(query_string, db="northwind-graph")])
dtf_data

,company,customer,order_count
0,Exotic Liquids,Felipe Izquierdo,5
1,Exotic Liquids,Jose Pavarotti,8
2,Exotic Liquids,Horst Kloss,6
3,New Orleans Cajun Delights,Paula Wilson,5
4,New Orleans Cajun Delights,Roland Mendel,6
5,Grandma Kelly's Homestead,Laurence Lebihan,5
6,Grandma Kelly's Homestead,Paula Wilson,5
7,Cooperativa de Quesos 'Las Cabras',Roland Mendel,5
8,Mayumi's,Jose Pavarotti,7
9,Mayumi's,Pirkko Koskitalo,5


***

## Task 9

Create a mechanism so that a customer can rate a product with a number between 1 and 5 (the input can be any customerID and any product
ID while there is no need for an input check for the rating). The rating should be stored as a property of a newly added relationship of type
:RATES PRODUCT. In order for the relationship to be added though,
two conditions must be met: a) the user has purchased the product in the
past at least once, and b) a user has not rated the product before.

In [22]:
query_string = "MATCH (n1:Product)<-[r1]-(n2:Order)<-[r2]-(n3:Customer) \
WHERE n1.productID = '4' and n3.customerID = '2' \
MERGE (n1)<-[:RATES_PRODUCT{rating: 4.5}]-(n3) \
RETURN r1, r2, n1, n2, n3"

In [23]:
dtf_data = pd.DataFrame([dict(_) for _ in conn.query(query_string, db="northwind-graph")])
dtf_data

""


***

## Task 10

 Create a mechanism so that a customer can rate a product with a supplier between 1 and 5 (the input can be any customerID and any supplier
ID while there is no need for an input check for the rating). The rating should be stored as a property of a newly added relationship of type
:RATES SHOP. Along with the rating, the user my add a text to describe
their experience. In order for the relationship to be added though, two
conditions must be met: a) the user has purchased at least one product
from the supplier, and b) a user has not rated the supplier before.

In [24]:
query_string = "MATCH (n1:Supplier)-[r]->(n2:Product)<-[r2]-(n3:Order)<-[r3]-(n4:Customer) \
WHERE n1.supplierID = '3' and n4.customerID = '2' \
MERGE (n1)<-[:RATES_SHOP{rating:2.5, text:'Alles super! Gerne wieder :)'}]-(n2)"

In [25]:
dtf_data = pd.DataFrame([dict(_) for _ in conn.query(query_string, db="northwind-graph")])
dtf_data

""
